# Heating Network Example

In the following a simple heat network with a [pump](https://pandapipes.readthedocs.io/en/latest/components/pump/pump_component.html) and a [heat exchanger](https://pandapipes.readthedocs.io/en/latest/components/heat_exchanger/heat_exchanger_component.html) is presented. In the case of the heat exchanger, the heat flow that is feed into the system from outside must be specified (*qext_w*). For the pump, a characteristic curve must be selected or newly defined. Here *qext_w* is set to 3000 and the pump characteristic curve "P1" is selected. Additionally, *add_stdtypes* must be set to True. The general aspects to consider when creating a pandapipes heating network can be found in the tutorial ["Creation of a pandapipes Heating Network"](https://github.com/e2nIEE/pandapipes/blob/master/tutorials/creating_a_heating_network.ipynb).

In [ ]:
import pandapipes

# create empty network
net = pandapipes.create_empty_network("net", add_stdtypes=True)

# create fluid
pandapipes.create_fluid_from_lib(net, "water", overwrite=True)

# create junctions
junction1 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=288.15, name="Junction 1", geodata=(0, 0))
junction2 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=288.15, name="Junction 2", geodata=(2, 0))
junction3 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=288.15, name="Junction 3", geodata=(2, 2))
junction4 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=288.15, name="Junction 4", geodata=(0, 2))

pandapipes.create_ext_grid(net, junction=junction1, p_bar=3, t_k=358.15, name="External Grid", type="pt")
pandapipes.create_sink(net, junction=junction3, mdot_kg_per_s=1, name="Sink")
pandapipes.create_source(net, junction=junction2, mdot_kg_per_s=3, name="Source")

# create pipes
pandapipes.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction2, length_km=1,
                                       diameter_m=0.07, k_mm=0.03, sections=4, alpha_w_per_m2k=50,
                                       text_k=303.15, name="Pipe 1", geodata=[(0, 0), (2, 0)])

pandapipes.create_pipe_from_parameters(net, from_junction=junction4, to_junction=junction1, length_km=3,
                                       diameter_m=0.05, k_mm=0.02, sections=7, alpha_w_per_m2k=150,
                                       text_k=303.15, name="Pipe 2", geodata=[(0, 2), (0, 0)])
# create pump
pandapipes.create_pump(net, from_junction=junction2, to_junction=junction3, std_type="P1", name="Pump")

# create heat exchanger
pandapipes.create_heat_exchanger(net, from_junction=junction3, to_junction=junction4, diameter_m=0.05,
                                 qext_w=3000, name="Heat Exchanger")

# run pipeflow
pandapipes.pipeflow(net, stop_condition="tol", iter=100, friction_model="nikuradse",
                    mode="all", transient=False, nonlinear_method="automatic", tol_p=1e-4,
                    tol_v=1e-4, ambient_temperature=303.15)

<img src="pics/heat_network_example.png">

The results of the calculations for the heat exchanger and the pump can be called up in the same way as for the junctions and pipes:

In [ ]:
net.res_heat_exchanger
net.res_pump